# 🧬 Training pipeline: Training ranking model </span>

In this notebook, you will train a ranking model using gradient boosted trees. 

In [33]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.filterwarnings("ignore")

from recsys.config import settings
from recsys.data.preprocessing.splitting import train_test_split
from recsys.gcp.vertex_ai.serving.ranking import GCPRankingModel
from recsys.core.models.two_tower.ranking import (
    RankingModelFactory,
    RankingModelTrainer,
)
from recsys.gcp.feature_store import client as fs_client
from recsys.gcp.bigquery import client as bq_client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## ☁️ Connect to Vertex AI Feature Online Store

In [11]:
fs_client.initialize()
fos = fs_client.get_client()

2025-02-19 15:12:11.343 | INFO     | recsys.gcp.feature_store.client:get_client:30 - Retrieving Feature Store from us-central1/recsys-dev-gonzo-2/recsys_feature_store_dev


## 💿 Create training dataset

In [12]:
trans_fv, articles_fv, customers_f, rankings_fv = fs_client.get_feature_views(fos)

In [16]:
rankings_df = bq_client.fetch_feature_view_data(feature_view=rankings_fv)
trans_df = bq_client.fetch_feature_view_data(feature_view=trans_fv)
articles_df = bq_client.fetch_feature_view_data(
    feature_view=articles_fv, except_columns=["embeddings"]
)

2025-02-19 16:08:10.676 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:181 - Fetching data from feature view: rankings
2025-02-19 16:08:11.010 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:194 - Executing query: SELECT * FROM `recsys-dev-gonzo-2.recsys_dataset.recsys_rankings`
2025-02-19 16:09:35.654 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:197 - DataFrame shape: (224136, 28)
2025-02-19 16:09:35.657 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:181 - Fetching data from feature view: transactions
2025-02-19 16:09:36.056 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:194 - Executing query: SELECT * FROM `recsys-dev-gonzo-2.recsys_dataset.recsys_transactions`
2025-02-19 16:09:39.592 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:197 - DataFrame shape: (23799, 11)
2025-02-19 16:09:39.593 | INFO     | recsys.gcp.bigquery.client:fetch_feature_view_data:181 - Fetching data from featur

In [17]:
rankings_df = rankings_df.join(
    trans_df.select(["customer_id", "month_sin", "month_cos"]),
    on="customer_id",
    how="left",
)

In [19]:
# rankings_df = rankings_df.rename({"perceived_colour_master_name": "colour_group_name"})
rankings_df.head(3)

customer_id,article_id,age,label,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,month_sin,month_cos
str,str,f64,i64,i64,str,i64,str,str,i64,str,str,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,i64,f64,f64
"""602a7857da345752e6a19c1575fd24…","""399061008""",22.0,0,399061,"""Jacket Slim""",262,"""Jacket""","""Garment Upper body""",1010023,"""Denim""","""71""","""Light Blue""",1,"""Dusty Light""",2,"""Blue""",1201,"""Outwear""","""A""","""Ladieswear""",1,"""Ladieswear""",19,"""Womens Jackets""",1007,"""Outdoor""",11,0.5,0.866025
"""602a7857da345752e6a19c1575fd24…","""399061008""",22.0,0,399061,"""Jacket Slim""",262,"""Jacket""","""Garment Upper body""",1010023,"""Denim""","""71""","""Light Blue""",1,"""Dusty Light""",2,"""Blue""",1201,"""Outwear""","""A""","""Ladieswear""",1,"""Ladieswear""",19,"""Womens Jackets""",1007,"""Outdoor""",11,0.5,0.866025
"""602a7857da345752e6a19c1575fd24…","""399061008""",22.0,0,399061,"""Jacket Slim""",262,"""Jacket""","""Garment Upper body""",1010023,"""Denim""","""71""","""Light Blue""",1,"""Dusty Light""",2,"""Blue""",1201,"""Outwear""","""A""","""Ladieswear""",1,"""Ladieswear""",19,"""Womens Jackets""",1007,"""Outdoor""",11,0.5,0.866025


In [34]:
X_train, X_val, y_train, y_val = train_test_split(
    df=rankings_df,
    test_size=settings.RANKING_DATASET_VALIDATION_SPLIT_SIZE,
)

2025-02-19 16:50:04.292 | INFO     | recsys.data.preprocessing.splitting:train_test_split:129 - Random split: train=16488920 rows (90.0%), test=1831067 rows (10.0%)


In [35]:
X_train.head(3)

customer_id,article_id,age,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,month_sin,month_cos
str,str,f64,i64,str,i64,str,str,i64,str,str,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,i64,f64,f64
"""602a7857da345752e6a19c1575fd24…","""399061008""",22.0,399061,"""Jacket Slim""",262,"""Jacket""","""Garment Upper body""",1010023,"""Denim""","""71""","""Light Blue""",1,"""Dusty Light""",2,"""Blue""",1201,"""Outwear""","""A""","""Ladieswear""",1,"""Ladieswear""",19,"""Womens Jackets""",1007,"""Outdoor""",11,0.5,0.866025
"""602a7857da345752e6a19c1575fd24…","""399061008""",22.0,399061,"""Jacket Slim""",262,"""Jacket""","""Garment Upper body""",1010023,"""Denim""","""71""","""Light Blue""",1,"""Dusty Light""",2,"""Blue""",1201,"""Outwear""","""A""","""Ladieswear""",1,"""Ladieswear""",19,"""Womens Jackets""",1007,"""Outdoor""",11,0.5,0.866025
"""602a7857da345752e6a19c1575fd24…","""399061008""",22.0,399061,"""Jacket Slim""",262,"""Jacket""","""Garment Upper body""",1010023,"""Denim""","""71""","""Light Blue""",1,"""Dusty Light""",2,"""Blue""",1201,"""Outwear""","""A""","""Ladieswear""",1,"""Ladieswear""",19,"""Womens Jackets""",1007,"""Outdoor""",11,0.5,0.866025


In [37]:
y_val.head(3)

label
i64
0
0
0


# Training the ranking model

Let's train the ranking model:

In [38]:
model = RankingModelFactory.build()

In [39]:
trainer = RankingModelTrainer(
    model=model, train_dataset=(X_train, y_train), eval_dataset=(X_val, y_val)
)

In [40]:
trainer.fit()

[0]	validation_0-logloss:0.59646
[1]	validation_0-logloss:0.53220
[2]	validation_0-logloss:0.48510
[3]	validation_0-logloss:0.44917
[4]	validation_0-logloss:0.42424
[5]	validation_0-logloss:0.39573
[6]	validation_0-logloss:0.37114
[7]	validation_0-logloss:0.35284
[8]	validation_0-logloss:0.33809
[9]	validation_0-logloss:0.32515
[10]	validation_0-logloss:0.31479
[11]	validation_0-logloss:0.30777
[12]	validation_0-logloss:0.29840
[13]	validation_0-logloss:0.29223
[14]	validation_0-logloss:0.28658
[15]	validation_0-logloss:0.28254
[16]	validation_0-logloss:0.27732
[17]	validation_0-logloss:0.27435
[18]	validation_0-logloss:0.26925
[19]	validation_0-logloss:0.26212
[20]	validation_0-logloss:0.25686
[21]	validation_0-logloss:0.24746
[22]	validation_0-logloss:0.24205
[23]	validation_0-logloss:0.23751
[24]	validation_0-logloss:0.23543
[25]	validation_0-logloss:0.23066
[26]	validation_0-logloss:0.22916
[27]	validation_0-logloss:0.22614
[28]	validation_0-logloss:0.22312
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=5,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

## Evaluating the ranking model

Next, you'll evaluate how well the model performs on the validation data using metrics for classification such as precision, recall and f1-score:

In [41]:
metrics = trainer.evaluate(log=True)

2025-02-19 16:55:26.703 | INFO     | recsys.core.models.two_tower.ranking:evaluate:113 -               precision    recall  f1-score   support

           0       1.00      0.97      0.99   1664111
           1       0.79      0.99      0.88    166956

    accuracy                           0.98   1831067
   macro avg       0.90      0.98      0.93   1831067
weighted avg       0.98      0.98      0.98   1831067



It can be seen that the model has a low F1-score on the positive class (higher is better). The performance could potentially be improved by adding more features to the dataset, e.g. image embeddings.

Let's see which features your model considers important.

In [42]:
trainer.get_feature_importance()

{'customer_id': 0.195463,
 'age': 0.14024463,
 'index_group_no': 0.054881696,
 'index_name': 0.046828866,
 'index_group_name': 0.043113835,
 'section_no': 0.04064512,
 'index_code': 0.038117964,
 'garment_group_name': 0.029079467,
 'section_name': 0.028327662,
 'garment_group_no': 0.027919007,
 'department_no': 0.027006269,
 'product_group_name': 0.026529534,
 'department_name': 0.02506187,
 'product_type_no': 0.02356282,
 'perceived_colour_master_id': 0.02234482,
 'graphical_appearance_no': 0.02206113,
 'perceived_colour_master_name': 0.021929612,
 'product_type_name': 0.021494182,
 'product_code': 0.020933306,
 'prod_name_length': 0.020141892,
 'graphical_appearance_name': 0.019884974,
 'prod_name': 0.019252827,
 'perceived_colour_value_name': 0.018567929,
 'colour_group_name': 0.017775314,
 'perceived_colour_value_id': 0.017699217,
 'colour_group_code': 0.016759275,
 'article_id': 0.014373878,
 'month_sin': 0.0,
 'month_cos': 0.0}

# 🗄️ Upload models to Vertex AI model registry

In [43]:
ranking_model_gcp = GCPRankingModel(model=model)

In [44]:
ranking_local_path = ranking_model_gcp.save_to_local("ranking_model")

In [45]:
ranking_model_vertex = ranking_model_gcp.upload_to_vertex_ai(
    model_name="ranking_model_v1",
    description="Ranking model for two-tower recommender system",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.2-1:latest",
)

2025-02-19 16:55:42.101 | INFO     | recsys.gcp.vertex_ai.model_registry:upload_model_to_registry:44 - Saving model into: /tmp/ranking_model_v1
2025-02-19 16:55:42.108 | INFO     | recsys.gcp.vertex_ai.model_registry:upload_model_to_registry:60 - Uploading model to ranking_model_v1 to Vertex AI


Creating Model
Create Model backing LRO: projects/434647834521/locations/us-central1/models/282087404687327232/operations/2645820697535840256
Model created. Resource name: projects/434647834521/locations/us-central1/models/282087404687327232@1
To use this Model in another session:
model = aiplatform.Model('projects/434647834521/locations/us-central1/models/282087404687327232@1')


2025-02-19 16:55:48.400 | INFO     | recsys.gcp.vertex_ai.model_registry:upload_model_to_registry:70 - Model uploaded with resource name: projects/434647834521/locations/us-central1/models/282087404687327232


In [46]:
rankind_endpoint = ranking_model_gcp.deploy_endpoint(
    model=ranking_model_vertex,
    endpoint_id="recsys-rankings-model",
)

2025-02-19 16:55:49.318 | INFO     | recsys.gcp.vertex_ai.serving.base:deploy_endpoint:106 - Found existing endpoint: recsys-rankings-model


Deploying model to Endpoint : projects/434647834521/locations/us-central1/endpoints/recsys-rankings-model
Deploy Endpoint model backing LRO: projects/434647834521/locations/us-central1/endpoints/recsys-rankings-model/operations/243150296333680640
Endpoint model deployed. Resource name: projects/434647834521/locations/us-central1/endpoints/recsys-rankings-model


2025-02-19 17:00:02.386 | INFO     | recsys.gcp.vertex_ai.serving.base:deploy_endpoint:122 - Model deployed to endpoint: projects/434647834521/locations/us-central1/endpoints/recsys-rankings-model
